<a href="https://colab.research.google.com/github/mvenouziou/Project-Text-Generation/blob/main/Mo_nonlinear_text_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Text Generation RNN

This program constructs a character-level sequence model to generate text according to a character distribution learned from the dataset. 

- Try my web app implementation at www.communicatemission.com/ml-projects#text_generation. (Currently, only the standard model is implemented in the app)
- See more at https://github.com/mvenouziou/Project-Text-Generation.

- See credits /attributions below

The code implements two different model architectures: "linear" and "nonlinear."
The linear model uses character-level embeddings to form the model. The nonlinear model adds a parallel word level embedding network, which is merged with the character embedding model. 

---

**What's New?**
*(Surely others have created similar models, however I personally have not seen them at the time of coding this. Citations are below for content I have seen elsewhere.)*

- Option to implement either the standard linear model architecture (see credits below) or nonlinear architectures.

- Nonlinear model architecture uses parallel RNN's for word-level embeddings and character-level embeddings. 

- Manage RNN statefulness for independent data sources. The linear model (credited below) codes' approach to statefulness imposes a dependence relation between samples / batches. This model implements the ability to treat independent works (individual poems, books, authors, etc.) as truly independent samples by resetting RNN states and shuffling independent data sources.

- Load and prepare data from multiple CSV and text files. Each rows from a CSV and each complete TXT file are treated as independent data sources. (CSV data prep accepts titles and content.) 

- Parameters to perturb learned probabilties in final generation function, to add extra variety to generated text.

---
**Credits / Citations / Attributions:**

**Linear Model and Shared Code** 

Other than items noted in previous sections, this python code and linear model structure is based heavily on Imperial College London's Coursera course, "Customising your models with Tensorflow 2" *(https://www.coursera.org/learn/customising-models-tensorflow2)* and the Tensorflow RNN text generation documentation *(https://www.tensorflow.org/tutorials/text/text_generation?hl=en).*


**Nonlinear Model:**   

This utilizes the pretrained embeddings:
-  Small BERT word embeddings from Tensorflow Hub, (*credited to Iulia Turc, Ming-Wei Chang, Kenton Lee, Kristina Toutanova's paper "Well-Read Students Learn Better: On the Importance of Pre-training Compact Models." *https://tfhub.dev/google/collections/bert/1)*
- ELECTRA-Small++ from Tensorflow Hub, (*credited to Kevin Clark and Minh-Thang Luong and Quoc V. Le and Christopher D. Manning's paper "ELECTRA: Pre-training Text Encoders as Discriminators Rather Than Generators." *https://hub.tensorflow.google.cn/google/electra_small/2)*

**Web App:** 

The web app is built on the Anvil platform and (at the time of this writing) is hosted on Google Cloud server (CPU).

**Datasets:**

- *'robert_frost_collection.csv'* is a Kaggle dataset available at https://www.kaggle.com/archanghosh/robert-frost-collection. Any other datasets used are public domain works available from Project Gutenberg https://www.gutenberg.org.

---

**About**

Find me online at:
- LinkedIn: https://www.linkedin.com/in/movenouziou/ 
- GitHub: https://github.com/mvenouziou

---

In [1]:
#### PACKAGE IMPORTS ####
# ML design
import tensorflow as tf
from tensorflow import keras
!pip install -q tensorflow-text
import tensorflow_text as text  # text processing / required for BERT encoder
import tensorflow_hub as hub  # for BERT encoder

# TensorBoard notebook extension
%load_ext tensorboard
import datetime, os

# data handling
import numpy as np
import pandas as pd
import string
import random
import re

# file management
import os
import bz2
import pickle
import _pickle as cPickle

In [2]:
# 3rd party integrations

# Mount Google Drive:
USE_GDRIVE = True
if USE_GDRIVE:
    GDRIVE_DIR = '/content/gdrive/'
    from google.colab import drive
    drive.mount(GDRIVE_DIR)

# Anvil's web app server
USE_ANVIL = False  
!pip install -q anvil-uplink
import anvil.server

if USE_ANVIL:
    anvil.server.connect('53NFXI7IX7IE233XQTVJDXUM-PUGRV2WON2LETWBG')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
# GLOBAL PARAMATERS

NUM_TRAILING_WORDS = 5  # for word model path
PADDED_EXAMPLE_LENGTH = 500  # for character model path
BATCH_SIZE = 32
USE_ELECTRA = True

In [4]:
# set author / file paths
AUTHOR = 'tests'

# model structure
USE_WORD_PATH = True
if USE_WORD_PATH:
    AUTHOR += '_words_model/'
if USE_ELECTRA:
    AUTHOR += 'electra/'
else: PATH_EXTENSION = ''

# saving models/ checkpoints
# (Google Drive)
FILEPATH = GDRIVE_DIR + 'MyDrive/Colab_Notebooks/models/text_generation/' + AUTHOR
CHECKPOINT_DIR = FILEPATH + '/checkpoints/'
PREDICTION_MODEL_DIR = FILEPATH + '/prediction_model/'
TRAINING_MODEL_DIR = FILEPATH + '/training_model/'
PROCESSED_DATA_DIR = FILEPATH + '/proc_data/'
TENSORBOARD_DIR = CHECKPOINT_DIR + '/logs/'
#os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

# online dataset repository
DATASETS_DIR = 'https://raw.githubusercontent.com/mvenouziou/text_generator/main/'
DATA_FILES = ['robert_frost_collection.csv']

### Define Encoders / Tokenizers

Character-Level

In [5]:
def create_character_tokenizer():
    """
    This function takes a list of strings as its argument. It should create 
    and return a Tokenizer according to the above specifications. 
    """
    
    char_tokens = string.printable
    filters = '#$%&()*+-/<=>@[]^_`{|}~\t'

    # Initialize standard keras tokenizer
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
                    num_words=None,  
                    filters=filters,
                    lower=False,  # conversion to lowercase letters
                    char_level=True,
                    oov_token=None,  # drop unknown characters
                    )
    
    # fit tokenizer
    tokenizer.fit_on_texts(char_tokens)

    return tokenizer

In [6]:
def make_padded_array(text_blocks, tokenizer=None, max_len=PADDED_EXAMPLE_LENGTH):
    # Tokenizes and applies padding for uniform length

    # load tokenizer if one is not supplied
    if tokenizer is None:
        tokenizer = create_character_tokenizer()

    # tokenize
    token_blocks = tokenizer.texts_to_sequences(text_blocks)

    # zero padding
    padded_blocks = tf.keras.preprocessing.sequence.pad_sequences(
                        sequences=token_blocks,  # dataset
                        maxlen=max_len, 
                        dtype='int32', 
                        padding='pre',
                        truncating='pre', 
                        value=0.0
                        )
    
    return padded_blocks

Word-Level (BERT or Electra pre-trained embedding)

In [7]:
def get_bert_encoder(seq_length=NUM_TRAILING_WORDS, use_electra=USE_ELECTRA):

    # Word Embeddings path (bert encoder)
    if use_electra:
        encoder_url = 'https://tfhub.dev/google/electra_small/2'
    else:
        encoder_url = 'https://tfhub.dev/tensorflow/' \
                            + 'small_bert/bert_en_uncased_L-2_H-128_A-2/1'
    preprocessor_url = 'https://tfhub.dev/tensorflow/' \
                        + 'bert_en_uncased_preprocess/3'
                
    # preprocessing layer
    # get BERT components
    preprocessor = hub.load(preprocessor_url)
    bert_tokenizer = hub.KerasLayer(preprocessor.tokenize,
                                    name='bert_tokenizer')
    bert_packer = hub.KerasLayer(preprocessor.bert_pack_inputs,
                                 arguments=dict(seq_length=seq_length),
                                 name='bert_input_packer')
    bert_encoder = hub.KerasLayer(encoder_url, trainable=False, 
                             name='BERT_encoder')
    
    return bert_tokenizer, bert_packer, bert_encoder

### Define Data Pre-processors

Load and Clean Datasets

In [8]:
# Function: loader for .csv files
def prepare_csv(filename, datasets_dir=DATASETS_DIR, 
                content_columns=['Name', 'Content'], shuffle_rows=True):
    
    # load data into DataFrame
    dataframe = pd.read_csv(datasets_dir + filename).dropna()
    
    # extract titles and content
    # note: column headings must match those below
    if 'Name ' in dataframe.columns:  # required for the Robert Frost set
        dataframe.rename(columns={'Name ':'Name'})
    
    # prepare titles
    try: 
        dataframe['Name'] = dataframe['Name'].apply(
                            lambda x: x.upper() + ':\n')
    except:
        # no titles found
        content_columns = ['Content']

    # prepare content
    dataframe['Content'] = dataframe['Content'].apply(
                    lambda x: x + '\n')

    # restrict dataset
    dataframe = dataframe[content_columns]

    # shuffle entries (rows)
    if shuffle_rows:
        dataframe = dataframe.sample(frac=1)
    
    # data cleanup
    dataframe = dataframe[content_columns]
    
    # merge desired text columns
    dataframe['merge'] = dataframe[content_columns[0]]
    for i in range(1, len(content_columns)):
        dataframe['merge'] = dataframe['merge'] + dataframe[content_columns[i]]

    # convert to list of strings
    data_list = dataframe['merge'].tolist()
    
    return data_list   


# Function: Load and standardize data files
def load_parse(data_list, display_samples=True):  

    # remove paragraph / line marks and split up words  
    tokenizer = text.WhitespaceTokenizer()

    # tokenize data (outputs bytestrings)
    cleaned_list_byte = [tokenizer.tokenize(data).numpy() for data in data_list]

    # convert data back to string format
    num_entries = len(cleaned_list_byte)

    clean_list = [' '.join(map(lambda x: x.decode(), cleaned_list_byte[i])) 
                    for i in range(num_entries)]

    # Display some text samples
    if display_samples:
        num_samples = 5
        inx = np.random.choice(len(clean_list), num_samples, replace=False)
        for example in np.array(clean_list)[inx]:
            print(example)
            print()

        print('len(text_chunks):', len(clean_list))

    return clean_list

In [9]:
def create_input_target_blocks(full_examples, tokenizer=None,
                               max_len=PADDED_EXAMPLE_LENGTH,
                               num_words=NUM_TRAILING_WORDS):
    # converts text into sliding n-grams of words and characters
    # returning input / target sets

    # helper function to create word-level inputs
    def update_word_char_lists(text, chars_list, words_list):
        words_input = text.split(' ')  # separate words
        words_input = words_input[-num_words-1: -1]  # get trailing words

        # convert words to string (tensor)
        words_input = ' '.join(words_input)

        # add values to lists
        chars_list.append(text)
        words_list.append([words_input])
        
        return None

    if tokenizer is None:
        tokenizer = create_character_tokenizer()

    blocks = []
    for example in full_examples:      

        char_block = []
        word_block = []
        example_length = len(example)

        # small blocks at start (will be zero-padded later)
        leading_characters = 1  # min chars to seed predictions
        for i in range(leading_characters, example_length - max_len - 1):
            text = example[: i]
            update_word_char_lists(text, char_block, word_block)

        # full length blocks
        for i in range(example_length - max_len - 1):
            # create n-gram
            text = example[i: max_len + i]
            update_word_char_lists(text, char_block, word_block)

        # small blocks at end (will be zero-padded later)
        for i in range(example_length - max_len - 1, example_length-1):
            text = example[i: ]
            update_word_char_lists(text, char_block, word_block)
    
        # tokenize and add pre-padding
        char_block = make_padded_array(char_block, tokenizer, max_len=max_len)

        # separate into inputs and targets
        inputs_char = char_block[:, :-1]
        targets_char = char_block[:, 1:]

        # update blocks
        word_block = np.array(word_block)
        blocks.append((inputs_char, word_block, targets_char))

    return blocks

In [10]:
# Function: data prep to create stateful RNN batches
# note: This will be applied separately on each example text, 
# so that RNN can reset internal state / distinguish between unrelated passages
# note: This code is taken directly from Imperial College London's 
# Coursera course cited above

def preprocess_stateful(char_input, word_input, target, batch_size=BATCH_SIZE):

    # Prepare input and output arrays for training the stateful RNN
    num_examples = char_input.shape[0]

    # adjust for batch size to divide evenly into sample size
    num_processed_examples = num_examples - (num_examples % batch_size)
    input_cropped = char_input[:num_processed_examples]
    target_cropped = target[:num_processed_examples]

    # separate out samples so rows of data match up across epochs
    # 'steps' measures how to space them out
    steps = num_processed_examples // batch_size  

    # define reordering
    inx = np.empty((0,), dtype=np.int32)  # initialize empty array object
    
    for i in range(steps):
        inx = np.concatenate((inx, i + np.arange(0, num_processed_examples, 
                                                    steps)))

    # reorder the data
    input_char_stateful = input_cropped[inx]
    input_word_stateful = word_input[inx]
    target_seq_stateful = target_cropped[inx]

    return input_char_stateful, input_word_stateful, target_seq_stateful

Input Pipeline

In [11]:
def input_pipeline(data_files=DATA_FILES, verbose=True, batch_size=BATCH_SIZE, 
                   max_len=PADDED_EXAMPLE_LENGTH, num_words=NUM_TRAILING_WORDS,
                   datasets_dir=DATASETS_DIR, saved_proc_dir=PROCESSED_DATA_DIR):

    # load previously processed data (pbz2 compressed file format)
    try:    
        with bz2.open(saved_proc_dir + 'datafiles.pbz2', 'rb') as file:
            data_dict = cPickle.load(file)

        X_data_list = data_dict['X_data_list']
        Y_data_list = data_dict['Y_data_list']

        print('loaded saved pre-processed data')

    except:       

        # load data file
        data_list = []
        for filename in data_files:

            # check file extension and select loader (csv or txt)
            _, file_extension = os.path.splitext(filename)     

            if file_extension == '.csv':   
                data = prepare_csv(filename, 
                                datasets_dir=datasets_dir, 
                                content_columns=['Name', 'Content'], 
                                shuffle_rows=True)
                
            else: # file_extension == '.txt':
                with open(filepath + '/' + filename, 'r', encoding='utf-8') as file:
                    data = file.readlines()

            # add extracted list of texts to data list
            data_list += data

        if verbose:
            print('PROGRESS: data_list created')
        
        # clean data
        clean_list = load_parse(data_list, display_samples=False)
        if verbose:
            print('PROGRESS: clean_list created')
        
        # preprocess data
        tokenizer = create_character_tokenizer()
        blocks = create_input_target_blocks(full_examples=clean_list, 
                                            tokenizer=tokenizer,
                                            max_len=max_len,
                                            num_words=num_words)
        if verbose:
            print('PROGRESS: blocks created')
        
        # create separate input / target pairs for each block
        X_data_list = []
        Y_data_list = []

        i=0
        for block in blocks:
            if i % 10 == 0:
                print(f'PROGRESS: processing block {i} of {len(blocks)}')

            char_input = block[0] 
            word_input = block[1] 
            target = block[2]

            input_char_stateful, input_word_stateful, target_seq_stateful = \
                                    preprocess_stateful(char_input=char_input, 
                                                        word_input=word_input, 
                                                        target=target, 
                                                        batch_size=batch_size)

            # group for model input
            X = [input_char_stateful, input_word_stateful]
            Y = target_seq_stateful

            X_data_list.append(X)
            Y_data_list.append(Y)

            # advance index
            i += 1

        # save file (pbz2 compressed file format)
        with bz2.BZ2File(saved_proc_dir + 'datafiles.pbz2', 'wb') as sfile:
            cPickle.dump({'X_data_list': X_data_list, 
                          'Y_data_list': Y_data_list}, sfile)

    return X_data_list, Y_data_list

### Define Models and Training Loop

Training Model

In [12]:
# Function: Model Definition
def get_training_model(use_word_path=USE_WORD_PATH,
                       verbose=True,
                       batch_size=BATCH_SIZE, 
                       padded_examples=PADDED_EXAMPLE_LENGTH,
                       num_words=NUM_TRAILING_WORDS,
                       use_electra=USE_ELECTRA):
    
    """ Defines and compiles our stateful RNN model. 
    Note: batch size is required argument for stateful RNN. """
    
    from keras.layers import Input, Embedding, Concatenate, Dense, GRU,\
                             Average, AveragePooling1D, Dropout, \
                             BatchNormalization, Lambda

    # parameters
    vocab_size = len(create_character_tokenizer().word_index) + 1
    embedding_dim = 128
    merge_dim = 128

    # pre-trained encoder
    bert_tokenizer, bert_packer, bert_encoder = \
            get_bert_encoder(use_electra=use_electra)
    
    # Build model
    # define input shapes
    input_1 = Input(shape=(None, ), #(padded_examples-1, ), 
                    batch_size=batch_size,
                    dtype=tf.int32, 
                    name='char_input')
    
    input_2 = Input(shape=(), 
                    batch_size=batch_size,
                    dtype=tf.string, 
                    name='word_input')

    # travel individual paths
    # Character Level Path
    # ## Char: Embedding
    x1 = Embedding(input_dim=vocab_size, output_dim=embedding_dim, 
                   mask_zero=True, batch_input_shape=(batch_size, None),
                   name='char_embedding',)(input_1)

    # ## Char: GRU 1
    x1 = GRU(units=embedding_dim, stateful=True, 
             return_sequences=True, name='char_GRU_1',)(x1)
    x1 = Dropout(rate=.10, name='char_Dropout_1')(x1)
    x1 = BatchNormalization(name='char_Batch_Norm_1')(x1)
    
    # ## Char: GRU Final --  must use output_dim = merge_dim!
    x1 = GRU(units=merge_dim, stateful=True, 
             return_sequences=True, name='char_GRU_final',)(x1)
    x1 = Dropout(rate=.10, name='char_Dropout_final')(x1)
    x1 = BatchNormalization(name='char_Batch_Norm_final')(x1)

    # Word Encoding Path
    if use_word_path:
        
        x2 = bert_tokenizer(input_2)  # tokenize
        x2 = bert_packer([x2])  # pack inputs for encoder
        x2 = bert_encoder(x2)['sequence_output'] # encoding

        # ## Word: GRU 1
        x2 = GRU(units=32, stateful=True, 
                 return_sequences=True, name='word_GRU_1',)(x2)
        x2 = Dropout(rate=.10, name='word_Dropout_1')(x2)
        x2 = BatchNormalization(name='word_Batch_Norm_1')(x2)

        # ## Word: Required conversion to valid merge output dim = merge_dim!
        x2 = Dense(units=num_words, activation=None, 
                   name='word_Dense_pre_final')(x2)
        x2 = AveragePooling1D(pool_size=5, padding='same', 
                              name='word_pooling_final')(x2)
        x2 = Dense(units=merge_dim, activation=None, 
                   name='word_Dense_final')(x2)

        # Merge Paths
        x = Average(name='merged_layers')([x1, x2])

    else:  # update variable id to match next step
        x = Lambda(lambda x: x, name='rename_variable')(x1)  
    
    # Final GRU layer
    x = GRU(units=embedding_dim, stateful=True, 
            return_sequences=True, name='GRU_OUTPUT')(x)          

    # Character prediction (logits)
    outputs = Dense(units=vocab_size, activation=None, 
                    name='Decoding')(x)       
    
    # create model
    model = keras.Model(inputs=[input_1, input_2], outputs=outputs)

    if verbose:
        print(model.summary())

    return model

Prediction Model

In [13]:
def get_prediction_model(trained_model=None, use_word_path=USE_WORD_PATH,
                         padded_examples=PADDED_EXAMPLE_LENGTH, verbose=False):
    """ enforces batch size = 1, only returns last character prediction
     and loads any saved weights """

    # set paramaters
    batch_size=1

    # create model
    prediction_model = get_training_model(batch_size=batch_size, 
                                          use_word_path=use_word_path,
                                          padded_examples=padded_examples,
                                          verbose=verbose)


    # load weights from pre-trained model
    if trained_model is not None:        
        trained_weights = trained_model.get_weights()
        prediction_model.set_weights(trained_weights)

    return prediction_model

Compiler

In [14]:
def compile_model(model, learning_rate):
    model.compile(optimizer=tf.keras.optimizers.Adamax(
                                learning_rate=learning_rate),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(
                                                    from_logits=True),
                  metrics=['sparse_categorical_accuracy', 
                        'sparse_categorical_crossentropy'],
                 )
    
    return model

Checkpoint Manager

In [15]:
# checkpoint manager
def create_checkpoint_manager(model, checkpoint_dir=CHECKPOINT_DIR):

    checkpoint = tf.train.Checkpoint(model=model)

    checkpoint_manager = tf.train.CheckpointManager(
                            checkpoint=checkpoint, 
                            directory=checkpoint_dir, 
                            max_to_keep=4, 
                            keep_checkpoint_every_n_hours=None,
                            checkpoint_name='ckpt', 
                            step_counter=None, 
                            checkpoint_interval=None,
                            init_fn=None
                            )
    
    return checkpoint, checkpoint_manager

Training Loop

In [16]:
# Function: Train model
def train_model(model, X_data_list, Y_data_list,
                num_epochs=1, 
                num_datasets_to_use=None,
                checkpoint=None, 
                checkpoint_manager=None,
                learning_rate=0.001,
                batch_size=BATCH_SIZE, 
                filepath=FILEPATH, 
                checkpoint_dir=CHECKPOINT_DIR,
                logdir=TENSORBOARD_DIR):

    # compile model
    model = compile_model(model, learning_rate=learning_rate)

    # set checkpoint manager
    if checkpoint is None or checkpoint_manager is None:
        checkpoint, checkpoint_manager = \
                    create_checkpoint_manager(model=model, 
                                              checkpoint_dir=checkpoint_dir)
                    
    # set callbacks
    # TensorBoard callback
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
                                log_dir=logdir, histogram_freq=1,
                                )
    
    # organize training data
    num_blocks = len(X_data_list)
    train_datasets_list = list(zip(X_data_list, Y_data_list)) 

    if num_datasets_to_use is None:  # if not specified, use all datasets
        num_datasets_to_use = len(X_data_list)      
    
    # begin training loop
    for epoch in range(num_epochs):

        print(f'Epoch: {epoch}')

        # shuffle dataset order
        random.shuffle(train_datasets_list)
        print('shuffled datasets')

        for i in range(num_datasets_to_use):
            print(f'dataset: {i}')

            # select dataset
            data = train_datasets_list[i]
            X = data[0]
            Y = data[1]

            # train model
            history = model.fit(x=X, y=Y,
                                shuffle=False,
                                epochs=1,
                                verbose=1,
                                #callbacks=[tensorboard_callback],
                                )
            
            # reset RNN hidden states
            model.reset_states()

            # save checkpoint
            checkpoint_manager.save()

    return model

### Define Implementation Functions


In [17]:
def convert_to_input(last_token, trunc_text, 
                     prepare_words=USE_WORD_PATH,
                     max_len=PADDED_EXAMPLE_LENGTH, 
                     num_words=NUM_TRAILING_WORDS):
    
    # words
    if prepare_words:
        words_input = trunc_text.split(' ')  # separate words
        words_input = words_input[-num_words-1:-1]  # get trailing words
        words_input = tf.constant(' '.join(trunc_text))  # convert to tensor
    else:
        words_input=tf.constant(' ')

    # pad token sequence
    inputs_char=tf.constant(last_token)
    """
    length = max_len - 1
    inputs_char = tf.keras.preprocessing.sequence.pad_sequences(
                        sequences=inputs_char,  # dataset
                        maxlen=length, 
                        dtype='int32', 
                        padding='pre',
                        truncating='pre', 
                        value=0.0
                        )
    """
    # create separate input / target pairs for each block
    X = [inputs_char, words_input]

    return X

In [18]:
def generator(input_text, prediction_model, precision_reduction=0, 
              num_characters=250, tokenizer=None, 
              max_len=PADDED_EXAMPLE_LENGTH, num_words=NUM_TRAILING_WORDS, 
              print_result=True):

    # get tokenizer (if not supplied)      
    if tokenizer is None:
        tokenizer = create_character_tokenizer()
    
    # initialize generated text
    last_token =  tokenizer.texts_to_sequences([input_text])
    trunc_text = input_text.upper() + ':\n'
    generated_text = []
   
    # text generation loop
    initial_state = None
    for _ in range(num_characters):

        # prepare input for model
        inputs = convert_to_input(last_token=last_token, 
                                  trunc_text=trunc_text,
                                  max_len=max_len,
                                  num_words=num_words)
        
        # pass forward final GRU layer state
        GRU_layer = prediction_model.get_layer('GRU_OUTPUT')
        GRU_layer.reset_states(initial_state)
        
        # run model and compute logits
        output = prediction_model(inputs)
        logits = output[:, -1, :]  # extract last character logits
        logits = logits.numpy()
       
        # generate next character from logits distribution
        # purturb probabilities (optional)
        if precision_reduction != 0:
            fuzz_factor = tf.random.normal(shape=logits.shape, mean=1, stddev=.2)
            logits = logits * (1 + precision_reduction * fuzz_factor)

        last_token = tf.random.categorical(logits=logits, num_samples=1)
        last_token = last_token.numpy().tolist()
        
        # get input for next character prediction
        input_text = tokenizer.sequences_to_texts(last_token)
        input_text = input_text[0]

        # record generated character
        generated_text.append(input_text)

        #  get GRU state for next character prediction
        initial_state = GRU_layer.states[0].numpy()

    # reset for next run
    output_text = ''.join(generated_text)
    
    if print_result:
        print(output_text)

    return output_text

Saving Models

In [19]:
# Store trained model separate from checkpoints
def save_model(model, model_dir):

    # save model
    model.save(model_dir)

    # get tokenizer
    prediction_tokenizer = create_character_tokenizer()
    
    # save tokenizer
    with open(model_dir + 'tokenizer.pickle', 'wb') as file:
        pickle.dump(prediction_tokenizer, file, pickle.HIGHEST_PROTOCOL)

    return None

# Implementation

Load and Process Data

In [20]:
try:
    if len(X_data_list) > 0:
        print('dataset already loaded')
except:
    print('Preparing dataset')
    X_data_list, Y_data_list = input_pipeline(DATA_FILES)

loaded saved pre-processed data


Initialize Training Model

In [21]:
training_model = get_training_model(verbose=True)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_input (InputLayer)         [(32,)]              0                                            
__________________________________________________________________________________________________
bert_tokenizer (KerasLayer)     (32, None, None)     0           word_input[0][0]                 
__________________________________________________________________________________________________
char_input (InputLayer)         [(32, None)]         0                                            
__________________________________________________________________________________________________
bert_input_packer (KerasLayer)  {'input_word_ids': ( 0           bert_tokenizer[0][0]             
______________________________________________________________________________________________

Load Latest Training Checkpoint

In [22]:
try:
    # load from checkpoint
    checkpoint, checkpoint_manager = \
        create_checkpoint_manager(model=training_model, 
                                    checkpoint_dir=CHECKPOINT_DIR)

    checkpoint_manager.restore_or_initialize()
    print('loaded checkpoint')

except:
    print('No matching checkpoints')

loaded checkpoint


Train Model

*Suggestion: overfitting can be a major problem where the model memorizes segments from the source material. Be careful to avoid this. A precision reduction factor of 0.0 - 1.5 can be used to partially compensate for this. (This paramater in my final prediction function randomly perturbs learned probabilities)*

In [ ]:
train_model_now=True
if train_model_now:
    num_epochs = 5

    training_model = train_model(training_model, X_data_list, Y_data_list,
                                num_epochs=num_epochs, 
                                num_datasets_to_use=None)  # note: num_datasets_to_use=None uses all of them

Epoch: 0
shuffled datasets
dataset: 0
30/30 [==============================] - 63s 2s/step - loss: 1.6342 - sparse_categorical_accuracy: 0.2655 - sparse_categorical_crossentropy: 4.0318
dataset: 1
60/60 [==============================] - 94s 2s/step - loss: 1.7425 - sparse_categorical_accuracy: 0.3567 - sparse_categorical_crossentropy: 3.4025
dataset: 2
19/19 [==============================] - 31s 2s/step - loss: 1.1666 - sparse_categorical_accuracy: 0.4033 - sparse_categorical_crossentropy: 3.3812
dataset: 3
15/15 [==============================] - 23s 2s/step - loss: 0.5867 - sparse_categorical_accuracy: 0.3648 - sparse_categorical_crossentropy: 3.1886
dataset: 4
15/15 [==============================] - 24s 2s/step - loss: 1.2378 - sparse_categorical_accuracy: 0.3547 - sparse_categorical_crossentropy: 3.2373
dataset: 5
120/120 [==============================] - 188s 2s/step - loss: 1.6303 - sparse_categorical_accuracy: 0.4587 - sparse_categorical_crossentropy: 3.1419
dataset: 6
55/55

Create Prediction Model

In [ ]:
prediction_model = get_prediction_model(trained_model=training_model)

Generate Text

In [ ]:
input_text = 'She '

generator(input_text=input_text, 
          prediction_model=prediction_model, 
          precision_reduction=0, 
          num_characters=250, 
          tokenizer=None, 
          max_len=PADDED_EXAMPLE_LENGTH, 
          num_words=NUM_TRAILING_WORDS, 
          print_result=True)

Save Models

In [ ]:
save_model_now = True

if save_model_now:
    # training model
    save_model(training_model, model_dir=TRAINING_MODEL_DIR)

    # prediction model
    save_model(prediction_model, model_dir=PREDICTION_MODEL_DIR)

Anvil Web App Server Integration

In [ ]:
# get tokenizer
tokenizer = create_character_tokenizer()

@anvil.server.callable
def generate_text(starting_text, precision_reduction,
                    prediction_tokenizer=tokenizer, 
                    prediction_model=prediction_model,
                    print_result=True,
                    author='assorted'):

    # set length of generated text
    num_generation_steps = 350

    # format user input
    starting_text = starting_text.upper() + ': '

    # get generated text
    # very rarely this produces too few line breaks, 
    # causing an indexing error. The exception below reruns it as needed
    while Tre:
        try:
            prediction = generator(input_text=starting_text, 
                                prediction_model=prediction_model, 
                                precision_reduction=precision_reduction, 
                                num_characters=num_generation_steps, 
                                tokenizer=prediction_tokenizer, 
                                max_len=PADDED_EXAMPLE_LENGTH, 
                                num_words=NUM_TRAILING_WORDS, 
                                print_result=print_result)        
            break
        except:
            pass                      
    
    output = starting_text + '\n'
    split_on = ['?', '.', ',', ';', '!', ':']
    splits = '([' + ''.join(split_on) + '])'
    split_lines_prediction = re.split(splits, prediction)
    
    for line in split_lines_prediction:
        line_update = line[0].upper()
        try: 
            line_update += line[1:]
        except:
            pass
        if line_update[-1] in split_on or line_update[-2:] == '\n':
            output= ''.join([output, line_update])
        else:
            output= '\n'.join([output, line_update])
    
    return output + '... '

if USE_ANVIL:
    # start persistent connection to server
    anvil.server.wait_forever()

In [ ]:
starting_text = 'She walks the line'
precision_reduction = .2

gen = generate_text(starting_text=starting_text, 
              precision_reduction=precision_reduction,
              print_result=False)

print(gen)
